<a href="https://colab.research.google.com/github/khalil-alexander/get_forecast/blob/main/FuturePositionISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <font color="red"> Pip install Astropy in google colab <font/>

In [ ]:
!pip install astropy

## <font color="red"> Install Moudles in google colab <font/>

In [ ]:
from astropy import coordinates as coord
from astropy import units as u
from astropy import time
from astropy.time import Time
from astropy.coordinates import Angle

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests as reqs
import xml.etree.ElementTree as ET
import io
import pprint
import json
import datetime

In [ ]:
from datetime import datetime
import datetime as dt
from datetime import timezone

In [ ]:
import xarray as xr

## <font color="red"> Converting coordinates to Latitude and Longitude ISS</font>

In [ ]:
now = Time("2024-03-07 00:46:00.000", scale='utc')
xyz = [1155.746046202530, -6632.420367726780, 953.533229633281]
cartrep = coord.CartesianRepresentation(*xyz, unit=u.km)



gcrs = coord.GCRS(cartrep, obstime = now)
itrs = gcrs.transform_to(coord.ITRS(obstime = now))
loc = coord.EarthLocation(*itrs.cartesian.xyz)

print(loc.lat, loc.lon, loc.height)

8d07m57.90103915s 103d26m20.85144985s 421.8445702007271 km


## <font color="red"> Creating functions to clean up Latitude and Longitude ISS</font>

### <font color="green">Create an empty dateframe for functions

### <font color="green">Create two function that changes km to lat/lon


In [ ]:
col_name = ['Date','x units(km)','y units(km)','z units(km)','x_dot(km/s)','y_dot(km/s)','z_dot(km/s)']
# Challenge to have the columns read under beautiful soup
xml_df = pd.DataFrame(columns=col_name)
xml_df

,Date,x units(km),y units(km),z units(km),x_dot(km/s),y_dot(km/s),z_dot(km/s)


In [ ]:
def create_latlon_positions(date: str, xyz: list[float]):
    now = Time(date, scale='utc')
    cartrep = coord.CartesianRepresentation(*xyz, unit=u.km)

    gcrs = coord.GCRS(cartrep, obstime = now)
    itrs = gcrs.transform_to(coord.ITRS(obstime = now))
    loc = coord.EarthLocation(*itrs.cartesian.xyz)

    return Angle(loc.lat).deg, Angle(loc.lon).deg


In [ ]:
def get_latlon(row: pd.Series):
    date = row['Date'].strftime('%Y-%m-%d %H:%M:%S')
    xyz = [row['x units(km)'], row['y units(km)'], row['z units(km)']]
    return create_latlon_positions(date, xyz)




## <font color="red"> read_xml_DateFrame
### <font color="green">A function that reads an xml file and returns the specfic position the iss is at during different time.

In [ ]:
def read_xml_DataFrame(url):
    """
    Do web scraping from an XML file to obtain time the International Space
    Station will be at a specfic postion and the speed it is going at.

    Parameters
    ----------
    url : String
        The url of the data in XML format you want to extract from.

    Return
    ------
    df : Pandas DataFrame
        The DataFrame containing the time the International Space Station is at
        a specfic position and the speed it is traveling.
    """
    # Get the xml request
    xml = reqs.get(url)

    # Use BeautifulSoup to Organize the XML file
    soup = BeautifulSoup(xml.content,"lxml")

    # Finds all the instances 'statevector' appeared in the xml file
    state_tag = soup.find_all('statevector')

    col_name = ['Date','x units(km)',
                'y units(km)','z units(km)',
                'x_dot(km/s)',
                'y_dot(km/s)','z_dot(km/s)']
    df = pd.DataFrame(columns=col_name)

    #Loops thorugh all elements to add rows in the empty xml_df
    for element in state_tag:
        # Turns element into text and splits the '\n'
        element = element.text.split('\n')
        # Deletes First empty String
        element.pop(0)
        # Deletes Last empty Sting
        element.pop(-1)
        # Removes the 'Z' in the elements first index
        element[0] = element[0][:-1]

        # Appending list into DataFrame by adding 1 row to the DataFrame
        #df = xml_df.append(pd.DataFrame(element, columns=col_name), ignore_index=True)
        df.loc[len(df)] = element


    # Turns the "Date" collumn into a Datetime object
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%jT%H:%M:%S.%f')
    # Sets the "Date" collumn into a Datetime object
    #df.set_index("Date", inplace=True)

    # Returns the Dataframe
    return df

In [ ]:
xml_url = "https://nasa-public-data.s3.amazonaws.com/iss-coords/current/ISS_OEM/ISS.OEM_J2K_EPH.xml"
%time df = read_xml_DataFrame(xml_url)

<ipython-input-92-ea97148a76b9>:21: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(xml.content,"lxml")


CPU times: user 13.3 s, sys: 82.7 ms, total: 13.4 s
Wall time: 20.8 s


In [ ]:
df

,Date,x units(km),y units(km),z units(km),x_dot(km/s),y_dot(km/s),z_dot(km/s)
0,2024-12-04 12:00:00,708.54449589064302,-4160.1722202186002,5320.8713077167904,7.5287662287788599,1.40720681202543,0.10122318940129001
1,2024-12-04 12:04:00,2467.78342080724,-3675.4542327102699,5151.0748935641404,7.0421770334086897,2.60745434190336,-1.5075036240923001
2,2024-12-04 12:08:00,4047.7634792311201,-2923.7756714666298,4606.0663030393098,6.0439934398579904,3.6183838626770801,-3.0065318411555602
3,2024-12-04 12:12:00,5333.6435354772102,-1959.6813399339801,3725.45645102037,4.6061700716427101,4.3667952550067497,-4.2871103933482297
4,2024-12-04 12:16:00,6231.7977688340497,-853.14089029681304,2573.2676276933998,2.8326093198761302,4.7981326452572199,-5.25577091906327
...,...,...,...,...,...,...,...
5398,2024-12-19 11:45:00,2187.6000218924,4191.6081359030804,-4880.04736465716,-4.8795023134572997,5.3728432592608097,2.43605979786923
5399,2024-12-19 11:49:00,951.30522322924605,5313.25769373195,-4124.8173124102104,-5.36027327116199,3.9168212996952598,3.81944169075916
5400,2024-12-19 11:53:00,-354.11836916283499,6048.6105864067904,-3068.8654902151602,-5.4518281968255504,2.1731454730044502,4.92658862855387
5401,2024-12-19 11:57:00,-1633.7132404823401,6343.4142871513004,-1788.78207090163,-5.1461291707180399,0.26799404899085999,5.67551267264421


#### <font color="reeee">Adds latitude and longitude to the Data Frame

In [ ]:
%time df[['lat','lon']] = df.apply(get_latlon, axis=1, result_type='expand')

CPU times: user 54.9 s, sys: 834 ms, total: 55.7 s
Wall time: 1min 3s


## <font color="red">Confrim if lat is below 50 and above -50
### <font color="green">We do this because the International Space Station does not travel past 50 and -50 degrees latitude.

### <font color="green">verify_latitude function
- <font color="blue">Returns the min and max latitude in a tuple so the observer can see if the latitude matches the International Space Station path.

In [ ]:
def check_latitude(df: pd.DataFrame):
    """
    Gives the minimum and maximum latitude of the path the ISS travels in a
    DataFrame.

    Parameters
    ----------
    df : Pandas Dataframe
        The url of the data in XML format you want to extract from.

    Return
    ------
    df : Tuple
        Returns df max and min latitude.
    """
    return df['lat'].min(), df['lat'].max()

In [ ]:
check_latitude(df)

(-51.79292157874869, 51.793363080300765)

In [ ]:
df = df[['Date','lat','lon']]

In [ ]:
df

,Date,lat,lon
0,2024-12-04 12:00:00,51.777990,26.017626
1,2024-12-04 12:04:00,49.576397,49.264901
2,2024-12-04 12:08:00,42.981169,68.601496
3,2024-12-04 12:12:00,33.545483,83.309694
4,2024-12-04 12:16:00,22.514467,94.709995
...,...,...,...
5398,2024-12-19 11:45:00,-46.019828,157.814892
5399,2024-12-19 11:49:00,-37.533683,174.243106
5400,2024-12-19 11:53:00,-27.014075,-173.224288
5401,2024-12-19 11:57:00,-15.398811,-163.101812


## <font color="red">Functions

### 1. <font color="green">obtain_future_date
- <font color="blue">A function where you take the have argurements Pandas Dataframe and nhours.
The function takes the current time and adds in by nhours which the default is three. You will pass in a Pandas dataframe that will take the future values between startdate and 24 hours ahead.

### 2. <font color="green">future_dates
 - <font color="blue">A function where you take the have argurements Pandas Dataframe. The function takes the current time and adds in by three hours.


In [ ]:
def obtain_future_date(df: pd.DataFrame, nhours: int=3):
    """
    Collects future position of the ISS with nhours from current time and 6
    hours ahead of nhours

    Parameters
    ----------
    df : Pandas Dataframe
        Dataframe which contains a column with datetime objects.

    nhours: Int
        Number of hours you want to start collecting data from current time.

    Return
    ------
    df : Pandas DataFrame
        The DataFrame containing the time the International Space Station
        from current time + nhours to 6 hours ahead.
    """
    # String of the current time of day now
    start_date = dt.datetime.now() + dt.timedelta(hours=nhours)
    #end_date = start_date + dt.timedelta(hours=24)
    # Turns the date from a string object to a datetime object
    start_date = start_date.strftime("%m/%d/%Y %H:%M:%S")
    start_date = pd.to_datetime(start_date, format='%m/%d/%Y %H:%M:%S')

    end_date = start_date + dt.timedelta(hours=6)
    #end_date = end_date.strftime("%m/%d/%Y %H:%M:%S")
    #end_date = pd.to_datetime(start_date, format='%m/%d/%Y %H:%M:%S')
    #return (start_date, end_date)

    return df[(start_date <= df['Date']) & (end_date >= df['Date'])]

In [ ]:
df1 = obtain_future_date(df)

In [ ]:
df1

,Date,lat,lon
880,2024-12-06 22:36:00,33.584197,155.820450
881,2024-12-06 22:40:00,43.022232,170.565960
882,2024-12-06 22:44:00,49.603547,-170.053713
883,2024-12-06 22:48:00,51.775797,-146.782583
884,2024-12-06 22:52:00,48.827574,-123.917592
...,...,...,...
965,2024-12-07 04:16:00,-49.922669,-31.413295
966,2024-12-07 04:20:00,-43.618047,-11.708848
967,2024-12-07 04:24:00,-34.351383,3.325114
968,2024-12-07 04:28:00,-23.404202,14.936997


## EXPLORE

In [ ]:
df1

,Date,lat,lon
880,2024-12-06 22:36:00,33.584197,155.820450
881,2024-12-06 22:40:00,43.022232,170.565960
882,2024-12-06 22:44:00,49.603547,-170.053713
883,2024-12-06 22:48:00,51.775797,-146.782583
884,2024-12-06 22:52:00,48.827574,-123.917592
...,...,...,...
965,2024-12-07 04:16:00,-49.922669,-31.413295
966,2024-12-07 04:20:00,-43.618047,-11.708848
967,2024-12-07 04:24:00,-34.351383,3.325114
968,2024-12-07 04:28:00,-23.404202,14.936997


In [ ]:
for element in df1['lon']:
    print(element)


155.82044964480446
170.56595959985054
-170.05371306120603
-146.78258274154157
-123.91759197254746
-105.3127486320441
-91.20148304929916
-80.16925020055146
-70.84642800318215
-62.17934935856738
-53.26949767530249
-43.15997206688229
-30.634968883930014
-14.207318118782268
6.956224679417683
30.663591813214516
52.10418197096499
68.83433457086636
81.57092717209335
91.80452609457281
100.77387088131532
109.4493651615273
118.73079244174744
129.66192200910774
143.59838842389425
161.976226775089
-175.29864809989303
-151.9298001839967
-132.32384060899452
-117.38816307068925
-105.84578501426881
-96.26634524595545
-87.53964732218907
-78.75254167635188
-68.97765081146326
-57.06549476016187
-41.558190881248834
-21.325829897878123
2.229825549681329
24.47139231648163
42.17740955814953
55.6131521518808
66.24941048261802
75.39383441367349
84.05753459216525
93.13721316387442
103.62988603682871
116.82220209414452
134.20290081385102
156.2112221630217
179.8759544235505
-159.54640454579578
-143.72443567504322

In [ ]:
df1.index = range(len(df1))

In [ ]:
lon_df = df1['lon']

for i in range(len(lon_df)-1):
    print(lon_df[i], lon_df[i+1] )

155.82044964480446 170.56595959985054
170.56595959985054 -170.05371306120603
-170.05371306120603 -146.78258274154157
-146.78258274154157 -123.91759197254746
-123.91759197254746 -105.3127486320441
-105.3127486320441 -91.20148304929916
-91.20148304929916 -80.16925020055146
-80.16925020055146 -70.84642800318215
-70.84642800318215 -62.17934935856738
-62.17934935856738 -53.26949767530249
-53.26949767530249 -43.15997206688229
-43.15997206688229 -30.634968883930014
-30.634968883930014 -14.207318118782268
-14.207318118782268 6.956224679417683
6.956224679417683 30.663591813214516
30.663591813214516 52.10418197096499
52.10418197096499 68.83433457086636
68.83433457086636 81.57092717209335
81.57092717209335 91.80452609457281
91.80452609457281 100.77387088131532
100.77387088131532 109.4493651615273
109.4493651615273 118.73079244174744
118.73079244174744 129.66192200910774
129.66192200910774 143.59838842389425
143.59838842389425 161.976226775089
161.976226775089 -175.29864809989303
-175.298648099893

In [ ]:
def count_orbit(df: pd.DataFrame):
    count = 0
    temp_df = df['lon']

    for i in range(len(temp_df)-1):
        if (temp_df[i] > 0) and (temp_df[i+1] < 0):
            count += 1
            print(temp_df[i], temp_df[i+1])
    return count

In [ ]:
orbit = count_orbit(df1)

170.56595959985054 -170.05371306120603
161.976226775089 -175.29864809989303
179.8759544235505 -159.54640454579578
173.01984593145804 -170.21779318358438


In [ ]:
orbit

4

In [ ]:
dt = datetime.now()
zero_minutes = datetime.min
print(zero_minutes)
td = dt - zero_minutes
total_seconds = td.total_seconds()
minutes = total_seconds / 60

print(minutes)

0001-01-01 00:00:00
1064485175.1785086


In [ ]:
# There are

orbit_in_minutes = total_seconds / 5400

print(orbit_in_minutes)

11827613.057538984


## xarray

In [ ]:
#type(dt.datetime.now() + dt.timedelta(hours=3))

In [ ]:
def changeto_xarray(df):
    """
    Using a previous DataFrame and converting it into an Xarray.

    Parameters
    ----------
    df : Pandas DataFrame



    Return
    ------
    ds : Xarray
        The Xarray containing the collected following data variables:
        Date, Latitude(lat), and Longitude(lon).

    """

    #Converts the hourly  DateTime object to an Xarray object which return a more percise time with it's weather conditions
    ds = df.to_xarray()

    return ds

In [ ]:
df1

,Date,lat,lon
0,2024-12-06 22:36:00,33.584197,155.820450
1,2024-12-06 22:40:00,43.022232,170.565960
2,2024-12-06 22:44:00,49.603547,-170.053713
3,2024-12-06 22:48:00,51.775797,-146.782583
4,2024-12-06 22:52:00,48.827574,-123.917592
...,...,...,...
85,2024-12-07 04:16:00,-49.922669,-31.413295
86,2024-12-07 04:20:00,-43.618047,-11.708848
87,2024-12-07 04:24:00,-34.351383,3.325114
88,2024-12-07 04:28:00,-23.404202,14.936997


In [ ]:
test_ds = changeto_xarray(df1)

In [ ]:
test_ds

<xarray.Dataset> Size: 3kB
Dimensions:  (index: 90)
Coordinates:
  * index    (index) int64 720B 0 1 2 3 4 5 6 7 8 ... 81 82 83 84 85 86 87 88 89
Data variables:
    Date     (index) datetime64[ns] 720B 2024-12-06T22:36:00 ... 2024-12-07T0...
    lat      (index) float64 720B 33.58 43.02 49.6 51.78 ... -34.35 -23.4 -11.58
    lon      (index) float64 720B 155.8 170.6 -170.1 ... 3.325 14.94 24.56

## test function

In [ ]:
test_date = "2024-12-06T20:16:00.00"

In [ ]:
test_datetwo = "2024-12-25 12:50"

In [ ]:
test_date = pd.to_datetime(test_date, format='%Y-%m-%dT%H:%M:%S.%f')

In [ ]:
test_datetwo = pd.to_datetime(test_datetwo, format='%Y-%m-%d %H:%M')

In [ ]:
test_ds.Date

<xarray.DataArray 'Date' (index: 90)> Size: 720B
array(['2024-12-06T22:36:00.000000000', '2024-12-06T22:40:00.000000000',
       '2024-12-06T22:44:00.000000000', '2024-12-06T22:48:00.000000000',
       '2024-12-06T22:52:00.000000000', '2024-12-06T22:56:00.000000000',
       '2024-12-06T23:00:00.000000000', '2024-12-06T23:04:00.000000000',
       '2024-12-06T23:08:00.000000000', '2024-12-06T23:12:00.000000000',
       '2024-12-06T23:16:00.000000000', '2024-12-06T23:20:00.000000000',
       '2024-12-06T23:24:00.000000000', '2024-12-06T23:28:00.000000000',
       '2024-12-06T23:32:00.000000000', '2024-12-06T23:36:00.000000000',
       '2024-12-06T23:40:00.000000000', '2024-12-06T23:44:00.000000000',
       '2024-12-06T23:48:00.000000000', '2024-12-06T23:52:00.000000000',
       '2024-12-06T23:56:00.000000000', '2024-12-07T00:00:00.000000000',
       '2024-12-07T00:04:00.000000000', '2024-12-07T00:08:00.000000000',
       '2024-12-07T00:12:00.000000000', '2024-12-07T00:16:00.000000000',
       '2024-12-07T00:20:00.000000000', '2024-12-07T00:24:00.000000000',
       '2024-12-07T00:28:00.000000000', '2024-12-07T00:32:00.000000000',
       '2024-12-07T00:36:00.000000000', '2024-12-07T00:40:00.000000000',
       '2024-12-07T00:44:00.000000000', '2024-12-07T00:48:00.000000000',
       '2024-12-07T00:52:00.000000000', '2024-12-07T00:56:00.000000000',
       '2024-12-07T01:00:00.000000000', '2024-12-07T01:04:00.000000000',
       '2024-12-07T01:08:00.000000000', '2024-12-07T01:12:00.000000000',
...
       '2024-12-07T02:04:00.000000000', '2024-12-07T02:08:00.000000000',
       '2024-12-07T02:12:00.000000000', '2024-12-07T02:16:00.000000000',
       '2024-12-07T02:20:00.000000000', '2024-12-07T02:24:00.000000000',
       '2024-12-07T02:28:00.000000000', '2024-12-07T02:32:00.000000000',
       '2024-12-07T02:36:00.000000000', '2024-12-07T02:40:00.000000000',
       '2024-12-07T02:44:00.000000000', '2024-12-07T02:48:00.000000000',
       '2024-12-07T02:52:00.000000000', '2024-12-07T02:56:00.000000000',
       '2024-12-07T03:00:00.000000000', '2024-12-07T03:04:00.000000000',
       '2024-12-07T03:08:00.000000000', '2024-12-07T03:12:00.000000000',
       '2024-12-07T03:16:00.000000000', '2024-12-07T03:20:00.000000000',
       '2024-12-07T03:24:00.000000000', '2024-12-07T03:28:00.000000000',
       '2024-12-07T03:32:00.000000000', '2024-12-07T03:36:00.000000000',
       '2024-12-07T03:40:00.000000000', '2024-12-07T03:44:00.000000000',
       '2024-12-07T03:48:00.000000000', '2024-12-07T03:52:00.000000000',
       '2024-12-07T03:56:00.000000000', '2024-12-07T04:00:00.000000000',
       '2024-12-07T04:04:00.000000000', '2024-12-07T04:08:00.000000000',
       '2024-12-07T04:12:00.000000000', '2024-12-07T04:16:00.000000000',
       '2024-12-07T04:20:00.000000000', '2024-12-07T04:24:00.000000000',
       '2024-12-07T04:28:00.000000000', '2024-12-07T04:32:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * index    (index) int64 720B 0 1 2 3 4 5 6 7 8 ... 81 82 83 84 85 86 87 88 89

In [ ]:
testing123 = test_ds.interp(Date=test_date)

ValueError: Dimensions {'Date'} do not exist. Expected one or more of FrozenMappingWarningOnValuesAccess({'index': 90})

In [ ]:
testing123 = test_ds.interp(coords=[('Date', [test_date, test_datetwo])]) #< test_ds.interp(date=test_datetw0)

In [ ]:
type(test_date)

In [ ]:
def obtain_interpolation(start_date: string, end_date: string, ds):
    start_date = pd.to_datetime(start_date, format='%Y-%m-%d %H:%M')
    end_date = pd.to_datetime(end_date, format='%Y-%m-%d %H:%M')

    var = ds.interp(time=start_date) < ds.interp(time=end_date)